In [3]:
import pandas as pd

In [1]:
from rdflib import Graph, URIRef, Literal, XSD, Namespace, RDF, BNode

In [4]:
FOAF = Namespace('http://xmlns.com/foaf/0.1/')
MYNS = Namespace('http://dsci558.org/myfakenamespace#')
SCHEMA = Namespace('https://schema.org/')

In [5]:
fitness_kg = Graph()
fitness_kg.bind('myns', MYNS)
fitness_kg.bind('foaf', FOAF)

### Load the required files

In [6]:
exercises = pd.read_csv("Data_KG_Creation/exercise.csv")

In [7]:
exercises.head()

,id,name,url,type,equipment,mechanics,force_type,difficulty,overview,instructions,secondary_muscle,group,muscle
0,https://www.muscleandstrength.com/exercises/pl...,plantar fascia lacrosse ball,https://www.muscleandstrength.com/exercises/pl...,smr,lacrosse ball,isolation,compression,intermediate,using a lacrosse ball to perform smr on your p...,"in a standing position, place the lacrosse bal...",['none'],plantar-fascia,['plantar fascia']
1,https://www.muscleandstrength.com/exercises/la...,landmine rotation,https://www.muscleandstrength.com/exercises/la...,strength,landmine,compound,isometric,intermediate,the landmine rotation is a variation of the ru...,assume an athletic position with your feet sli...,['abs'],abs,['obliques']
2,https://www.muscleandstrength.com/exercises/ta...,tall kneeling cable lift,https://www.muscleandstrength.com/exercises/ta...,strength,cable,compound,isometric,beginner,the tall kneeling cable lift is a variation of...,assume a tall kneeling position and attach a r...,['abs'],abs,['obliques']
3,https://www.muscleandstrength.com/exercises/si...,side plank with hip dip,https://www.muscleandstrength.com/exercises/si...,strength,bodyweight,compound,isometric,beginner,the side plank with hip dip is a variation of ...,assume a side lying position with your forearm...,['abs'],abs,['obliques']
4,https://www.muscleandstrength.com/exercises/wo...,wood chop,https://www.muscleandstrength.com/exercises/wo...,strength,cable,compound,isometric,beginner,the standing cable chop is an exercise used to...,assume an athletic position with your feet jus...,['abs'],abs,['obliques']


In [ ]:
def createNodes(exercise):
    book_uri = URIRef(MYNS[str(exercise['ID'])])
    
    fitness_kg.add((book_uri))

In [ ]:
def createRecords(book):
    book_uri = URIRef(MYNS[str(book['ID'])])
    
    my_final_kg.add((book_uri, RDF.type, SCHEMA["Book"]))
    my_final_kg.add((book_uri, SCHEMA["name"], Literal(book["Title"])))
    
    my_final_kg.add((book_uri, SCHEMA["abstract"], Literal(book['Description'])))
    
    isbnNode = BNode()
    my_final_kg.add((isbnNode, RDF.type, SCHEMA["isbn"]))
    my_final_kg.add((book_uri, SCHEMA["isbn"], isbnNode))
    if len(book['ISBN13_GR'].strip())>0:
        my_final_kg.add((isbnNode, SCHEMA["isbn"], Literal(book['ISBN13_GR'], datatype = XSD.integer)))
    if len(book['ISBN13_BN'].strip())>0:
        my_final_kg.add((isbnNode, SCHEMA["isbn"], Literal(book['ISBN13_BN'], datatype = XSD.integer)))
    
    
    my_final_kg.add((book_uri, SCHEMA["numberOfPages"], Literal(book['PageCount'], datatype = XSD.integer)))
    
    authorNode = BNode()
    my_final_kg.add((authorNode, RDF.type, SCHEMA["author"]))
    my_final_kg.add((book_uri, SCHEMA["author"], authorNode))
    if len(book['FirstAuthor'].strip())>0:
        my_final_kg.add((authorNode, SCHEMA["author"], Literal(book['FirstAuthor'])))
    if len(book['SecondAuthor'].strip())>0:
        my_final_kg.add((authorNode, SCHEMA["author"], Literal(book['SecondAuthor'])))
    if len(book['ThirdAuthor'].strip())>0:
        my_final_kg.add((authorNode, SCHEMA["author"], Literal(book['ThirdAuthor'])))
    
    ratingNode = BNode()
    my_final_kg.add((ratingNode, RDF.type, SCHEMA["aggregateRating"]))
    my_final_kg.add((book_uri, SCHEMA["aggregateRating"], ratingNode))
    my_final_kg.add((ratingNode, SCHEMA["aggregateRating"], Literal(book['Rating'], datatype = XSD.float)))
    my_final_kg.add((ratingNode, SCHEMA["RatingCount"], Literal(book['NumberofRatings'], datatype = XSD.integer))) #
#     num_reviews = book['NumberofReviews'].replace(",", '')
    my_final_kg.add((ratingNode, SCHEMA["commentCount"], Literal(book['NumberofReviews'].replace(",", ""), datatype = XSD.integer))) #
    
    publisherNode = BNode()
    my_final_kg.add((publisherNode, RDF.type, SCHEMA["publisher"]))
    my_final_kg.add((book_uri, SCHEMA["publisher"], publisherNode))
    my_final_kg.add((publisherNode, SCHEMA["publisher"], Literal(book['Publisher_GR'])))
    my_final_kg.add((publisherNode, SCHEMA["publisher"], Literal(book['Publisher_BN'])))
    
    if len(book['PublicationDate'].strip())>0:
        my_final_kg.add((book_uri, SCHEMA["datePublished"], Literal(book['PublicationDate'], datatype = XSD.date)))
        
    my_final_kg.add((book_uri, SCHEMA["bookFormat"], Literal(book['Format']))) #, datatype = XSD.BookFormatType
    my_final_kg.add((book_uri, SCHEMA["inLanguage"], Literal(book['Language'])))
    my_final_kg.add((book_uri, SCHEMA["Rank"], Literal(book['Salesrank'])))
    my_final_kg.add((book_uri, SCHEMA["size"], Literal(book['Productdimensions'])))
    
    priceNode = BNode()
    my_final_kg.add((priceNode, RDF.type, SCHEMA["price"]))
    my_final_kg.add((book_uri, SCHEMA["price"], priceNode))
    if len(book['Paperbackprice'].strip())>0:
        my_final_kg.add((priceNode, SCHEMA["price"], Literal(book['Paperbackprice'])))
    if len(book['Hardcoverprice'].strip())>0:
        my_final_kg.add((priceNode, SCHEMA["price"], Literal(book['Hardcoverprice'])))
    if len(book['Nookbookprice'].strip())>0:
        my_final_kg.add((priceNode, SCHEMA["price"], Literal(book['Nookbookprice'])))
    if len(book['Audiobookprice'].strip())>0:
        my_final_kg.add((priceNode, SCHEMA["price"], Literal(book['Audiobookprice'])))

In [ ]:
i = 0
for _, bookDetails in result_deduplicate.iterrows():
    createRecords(bookDetails)

In [ ]:
my_final_kg.serialize(dir_ + 'Output\ShyamKrishnan_OndanatVeetil_model.ttl', format="turtle")

In [ ]:
!head 'Output\ShyamKrishnan_OndanatVeetil.ttl'